In [1]:
import json
import sys
import requests
from datetime import datetime
from dateutil import tz
import pprint

In [2]:
def fetch_data(server, year):
    data = requests.get('http://' + server + '/nhlplayoffs/api/v3.0/' + str(year) + '/data').json()
    return data

def update_data(server, year, data):
    url = 'http://' + server + '/nhlplayoffs/api/v3.0/' + str(year) + '/data'
    headers = {'content-type': 'application/json'}
    requests.post(url, data = json.dumps(data), headers=headers)

In [3]:
def get_team(id):
    url = 'https://statsapi.web.nhl.com/api/v1/teams/' + str(id)
    team = requests.get(url).json()
    return team['teams'][0]

def get_teams(year):
    ystr = str(year) + str(year+1)
    url = 'https://statsapi.web.nhl.com/api/v1/standings?season=' + ystr
    standings = requests.get(url).json()
    teams = {}
    for record in standings["records"]:
        for team in record['teamRecords']:
            info = get_team(team['team']['id'])
            team_record = {'info':info, 'standings':team, 'schedule':[]}
            teams[team['team']['id']] = team_record
    return teams

def get_standings(teams):
    standings = {'Eastern':{'Atlantic':[], 'Metropolitan':[], 'teams':[]},
                 'Western':{'Central':[], 'Pacific':[], 'teams':[]},
                 'teams':[]}

    league = sorted(teams, key=lambda k: int(k['standings']['divisionRank']))
    for team in league:
        standings['teams'].append(team)
        standings[team['info']['conference']['name']]['teams'].append(team)
        standings[team['info']['conference']['name']][team['info']['division']['name']].append(team)
    standings['teams'] = sorted(standings['teams'], key=lambda k: int(k['standings']['leagueRank']))

    standings['Eastern']['teams'] = sorted(standings['Eastern']['teams'], key=lambda k: int(k['standings']['conferenceRank']))
    standings['Western']['teams'] = sorted(standings['Western']['teams'], key=lambda k: int(k['standings']['conferenceRank']))

    standings['Eastern']['Atlantic'] = sorted(standings['Eastern']['Atlantic'], key=lambda k: int(k['standings']['divisionRank']))
    standings['Eastern']['Metropolitan'] = sorted(standings['Eastern']['Metropolitan'], key=lambda k: int(k['standings']['divisionRank']))
    standings['Western']['Central'] = sorted(standings['Western']['Central'], key=lambda k: int(k['standings']['divisionRank']))
    standings['Western']['Pacific'] = sorted(standings['Western']['Pacific'], key=lambda k: int(k['standings']['divisionRank']))

    return standings

def get_schedule(team, year):
    print('Get schedule for ' + str(team))
    url = 'https://statsapi.web.nhl.com/api/v1/schedule?startDate=' + str(year) + '-10-01&endDate=' + str(year+1) + '-06-29&expand=schedule.teams,schedule.linescore,schedule.broadcasts,schedule.ticket,schedule.game.content.media.epg&leaderCategories=&site=en_nhlCA&teamId=' + str(team)
    team_schedule = requests.get(url)
    return team_schedule.json()

def get_playoff_schedule(team, year):
    url = 'https://statsapi.web.nhl.com/api/v1/schedule?startDate=' + str(year+1) + '-04-01&endDate=' + str(year+1) + '-06-29&expand=schedule.teams,&site=en_nhlCA&teamId=' + str(team)
    team_schedule = requests.get(url)
    return team_schedule.json()

In [4]:
def set_matchup_childs(matchup, right, left):
    matchup['left'] = left
    matchup['right'] = right
    
def create_matchup(id, round, next):
    matchup = {'id': id, 'home':0, 'away':0, 'round':round, 'start':'', 'result':{}, 'schedule':[], 'season':{}, 'next': next}
    matchup['left'] = None
    matchup['right'] = None
    matchup['result'] = {'home_win':0, 'away_win':0}
    return matchup

def create_matchups_tree():
    matchups = {}
    sc = create_matchup('sc', 4, None)
    matchups[sc['id']] = sc
    
    e = create_matchup('e', 3, sc)
    w = create_matchup('w', 3, sc)
    matchups[e['id']] = e
    matchups[w['id']] = w
    
    a = create_matchup('a', 2, e)
    m = create_matchup('m', 2, e)
    c = create_matchup('c', 2, w)
    p = create_matchup('p', 2, w)
    matchups[a['id']] = a
    matchups[m['id']] = m
    matchups[c['id']] = c
    matchups[p['id']] = p
    
    a1 = create_matchup('a1', 1, a)
    a2 = create_matchup('a2', 1, a)
    m1 = create_matchup('m1', 1, m)
    m2 = create_matchup('m2', 1, m)
    c1 = create_matchup('c1', 1, c)
    c2 = create_matchup('c2', 1, c)
    p1 = create_matchup('p1', 1, p)
    p2 = create_matchup('p2', 1, p)
    matchups[a1['id']] = a1
    matchups[a2['id']] = a2
    matchups[m1['id']] = m1
    matchups[m2['id']] = m2
    matchups[c1['id']] = c1
    matchups[c2['id']] = c2
    matchups[p1['id']] = p1
    matchups[p2['id']] = p2
    
    #build tree
    set_matchup_childs(sc, e, w)
    
    set_matchup_childs(w, p, c)
    set_matchup_childs(e, m, a)
    
    set_matchup_childs(c, c2, c1)
    set_matchup_childs(p, p2, p1)
    set_matchup_childs(a, a2, a1)
    set_matchup_childs(m, m2, m1)
    
    return matchups

In [5]:
def get_round_matchups(matchups, round):
    results = []
    for matchup in list(matchups.values()):
        if matchup['round'] == round:
            results.append(matchup)
    return results

In [6]:
year = 2015
teams = get_teams(year)
standings = get_standings(list(teams.values()))
matchups = create_matchups_tree()

In [7]:
def parse_time(timestamp):
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/New_York')
    utc = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%SZ')
    utc = utc.replace(tzinfo=from_zone)
    return utc.astimezone(to_zone)

def get_matchup_schedule(matchup, year, schedules=None):
    home_id = matchup['home']
    away_id = matchup['away']
    result = []
    s = schedules
    if schedules is None:
        s = get_playoff_schedule(int(home_id), year)
    for date in s['dates']:
        game = date['games'][0]
        game_home_id = game['teams']['home']['team']['id']
        game_away_id = game['teams']['away']['team']['id']
        if game['gameType'] == 'P':
            if game_home_id == away_id or game_away_id == away_id:
                result.append(game)
    result = sorted(result, key=lambda k: parse_time(k['gameDate']))
    return result

def get_matchup_start(matchup):
    if len(matchup['schedule']) == 0:
        return ''
    return matchup['schedule'][0]['gameDate']

def get_matchup_result(matchup):
    result = {}
    home_id = matchup['home']
    away_id = matchup['away']
    home_win = 0
    away_win = 0
    for game in matchup['schedule']:
        game_home_id = game['teams']['home']['team']['id']
        game_away_id = game['teams']['away']['team']['id']
        if game['gameType'] == 'P':
            if game_home_id == away_id or game_away_id == away_id:
                if game_home_id == away_id: #reverse
                    away_score = game['teams']['home']['score']
                    home_score = game['teams']['away']['score']
                else:
                    away_score = game['teams']['away']['score']
                    home_score = game['teams']['home']['score']
                if home_score > away_score:
                    home_win = home_win + 1
                elif home_score < away_score:
                    away_win = away_win + 1
                #print(match['home']['team']['name'], home_score, match['away']['team']['name'], away_score)
                #print(game['teams']['home']['team']['name'],game['teams']['home']['score'],match['away']['team']['name'],game['teams']['away']['score'])
    result['home_win'] = home_win
    result['away_win'] = away_win
    return result
    
def is_matchup_finished(matchup):
    return matchup['result']['home_win'] == 4 or matchup['result']['away_win'] == 4

def get_matchup_winner(matchup):
    if matchup['result']['home_win'] == 4:
        return matchup['home']
    if matchup['result']['away_win'] == 4:
        return matchup['away']
    return 0

def update_matchup(matchup, home=0, away=0):
    if is_matchup_finished(matchup):
        return

    if matchup['home'] != 0 and matchup['away'] != 0:
        #update result and maybe pass to next stage
        matchup['schedule'] = get_matchup_schedule(matchup, year)
        if matchup['start'] == '':
            matchup['start'] = get_matchup_start(matchup)
        matchup['result'] = get_matchup_result(matchup)
        if is_matchup_finished(matchup) and matchup['next'] is not None:
            print('Finished',matchup['id'])
            update_matchup(matchup['next'], get_matchup_winner(matchup))
    else:
        if matchup['home'] == 0:
            matchup['home'] = home
            matchup['away'] = away
        else:
            matchup['away'] = home
            
        if matchup['home'] != 0 and matchup['away'] != 0:
            #Begin matchup
            hi = teams[matchup['home']]
            ai = teams[matchup['away']]
            if int(hi['standings']['leagueRank']) > int(ai['standings']['leagueRank']): 
                matchup['home'] = ai['info']['id']
                matchup['away'] = hi['info']['id']
            hi = teams[matchup['home']]
            ai = teams[matchup['away']]
            matchup['schedule'] = get_matchup_schedule(matchup, year)
            matchup['start'] = get_matchup_start(matchup)

def update_matchups(matchups):
    ms = list(matchups.values())
    ms = sorted(ms, key=lambda k: k['round'])
    for matchup in ms:
        update_matchup(matchup)

In [8]:
ealeader = standings['Eastern']['Atlantic'][0]
emleader = standings['Eastern']['Metropolitan'][0]
wcleader = standings['Western']['Central'][0]
wpleader = standings['Western']['Pacific'][0]
for team in standings['Eastern']['teams']:
    if int(team['standings']['wildCardRank']) == 1:
        e1wild = team
    if int(team['standings']['wildCardRank']) == 2:
        e2wild = team

for team in standings['Western']['teams']:
    if int(team['standings']['wildCardRank']) == 1:
        w1wild = team
    if int(team['standings']['wildCardRank']) == 2:
        w2wild = team

a1 = matchups['a1']
m1 = matchups['m1']
update_matchup(a1, ealeader['info']['id'])
update_matchup(m1, emleader['info']['id'])
if int(ealeader['standings']['conferenceRank']) < int(emleader['standings']['conferenceRank']):
    update_matchup(a1, e2wild['info']['id'])
    update_matchup(m1, e1wild['info']['id'])
else:
    update_matchup(a1, e1wild['info']['id'])
    update_matchup(m1, e2wild['info']['id'])
    
a2 = matchups['a2']
update_matchup(a2, standings['Eastern']['Atlantic'][1]['info']['id'], standings['Eastern']['Atlantic'][2]['info']['id'])
m2 = matchups['m2']
update_matchup(m2, standings['Eastern']['Metropolitan'][1]['info']['id'], standings['Eastern']['Metropolitan'][2]['info']['id'])

c1 = matchups['c1']
p1 = matchups['p1']
update_matchup(c1, wcleader['info']['id'])
update_matchup(p1, wpleader['info']['id'])
if int(wcleader['standings']['conferenceRank']) < int(wpleader['standings']['conferenceRank']):
    update_matchup(c1, w2wild['info']['id'])
    update_matchup(p1, w1wild['info']['id'])
else:
    update_matchup(c1, w1wild['info']['id'])
    update_matchup(p1, w2wild['info']['id'])
    
c2 = matchups['c2']
update_matchup(c2, standings['Western']['Central'][1]['info']['id'], standings['Western']['Central'][2]['info']['id'])
p2 = matchups['p2']
update_matchup(p2, standings['Western']['Pacific'][1]['info']['id'], standings['Western']['Pacific'][2]['info']['id'])


In [14]:
matchups = {}
if int(ealeader['standings']['conferenceRank']) < int(emleader['standings']['conferenceRank']):
    a1 = {'id': 'a1', 'home': ealeader['info']['id'], 'away': e2wild['info']['id'], 'round':1}
    a2 = {'id': 'a2', 'home': standings['Eastern']['Atlantic'][1]['info']['id'], 'away': standings['Eastern']['Atlantic'][2]['info']['id'], 'round':1}
    m1 = {'id': 'm1', 'home': emleader['info']['id'], 'away': e1wild['info']['id'], 'round':1}
    m2 = {'id': 'm2', 'home': standings['Eastern']['Metropolitan'][1]['info']['id'], 'away': standings['Eastern']['Metropolitan'][2]['info']['id'], 'round':1}
    
elif int(ealeader['standings']['conferenceRank']) > int(emleader['standings']['conferenceRank']):
    a1 = {'id': 'a1', 'home': ealeader['info']['id'], 'away': e1wild['info']['id'], 'round':1}
    a2 = {'id': 'a2', 'home': standings['Eastern']['Atlantic'][1]['info']['id'], 'away': standings['Eastern']['Atlantic'][2]['info']['id'], 'round':1}
    m1 = {'id': 'm1', 'home': emleader['info']['id'], 'away': e2wild['info']['id'], 'round':1}
    m2 = {'id': 'm2', 'home': standings['Eastern']['Metropolitan'][1]['info']['id'], 'away': standings['Eastern']['Metropolitan'][2]['info']['id'], 'round':1}

a = {'id': 'a', 'home':0, 'away':0, 'round':2}
a1['next'] = a
a2['next'] = a

m = {'id': 'm', 'home':0, 'away':0, 'round':2}
m1['next'] = m
m2['next'] = m

e = {'id': 'e', 'home':0, 'away':0, 'round':3}
a['next'] = e
m['next'] = e

matchups[a1['id']] = a1
matchups[a2['id']] = a2
matchups[m1['id']] = m1
matchups[m2['id']] = m2
matchups[a['id']] = a
matchups[m['id']] = m
matchups[e['id']] = e

In [17]:
if int(wcleader['standings']['conferenceRank']) < int(wpleader['standings']['conferenceRank']):
    c1 = {'id': 'c1', 'home': wcleader['info']['id'], 'away': w2wild['info']['id'], 'round':1}
    c2 = {'id': 'c2', 'home': standings['Western']['Central'][1]['info']['id'], 'away': standings['Western']['Central'][2]['info']['id'], 'round':1}
    p1 = {'id': 'p1', 'home': wpleader['info']['id'], 'away': w1wild['info']['id'], 'round':1}
    p2 = {'id': 'p2', 'home': standings['Western']['Pacific'][1]['info']['id'], 'away': standings['Western']['Pacific'][2]['info']['id'], 'round':1}

elif int(wcleader['standings']['conferenceRank']) > int(wpleader['standings']['conferenceRank']):
    c1 = {'id': 'c1', 'home': wcleader['info']['id'], 'away': w1wild['info']['id'], 'round':1}
    c2 = {'id': 'c2', 'home': standings['Western']['Central'][1]['info']['id'], 'away': standings['Western']['Central'][2]['info']['id'], 'round':1}
    p1 = {'id': 'p1', 'home': wpleader['info']['id'], 'away': w2wild['info']['id'], 'round':1}
    p2 = {'id': 'p2', 'home': standings['Western']['Pacific'][1]['info']['id'], 'away': standings['Western']['Pacific'][2]['info']['id'], 'round':1}
    
c = {'id': 'c', 'home':0, 'away':0, 'round':2}
c1['next'] = c
c2['next'] = c

p = {'id': 'p', 'home':0, 'away':0, 'round':2}
p1['next'] = p
p2['next'] = p

w = {'id': 'w', 'home':0, 'away':0, 'round':3}
c['next'] = w
p['next'] = w

matchups[c1['id']] = c1
matchups[c2['id']] = c2
matchups[p1['id']] = p1
matchups[p2['id']] = p2
matchups[c['id']] = c
matchups[p['id']] = p
matchups[w['id']] = w

sc = {'id': 'sc', 'home':0, 'away':0, 'round':4}
e['next'] = sc
w['next'] = sc

matchups[sc['id']] = sc

In [92]:
def build_matchup_tree(raw_matchups):
    matchups = {}
    for matchup_raw in list(raw_matchups.values()):
        matchup = matchup_raw.copy()
        matchups[matchup['id']] = matchup
        
    for matchup in list(matchups.values()):
        next = matchup['next']
        right = matchup['right']
        left = matchup['left']
        if next in raw_matchups:
            matchup['next'] = matchups[next]
        if right in raw_matchups:
            matchup['right'] = matchups[right]
        if left in raw_matchups:
            matchup['left'] = matchups[left]
    return matchups

def store_matchup_tree(matchups):
    raw_matchups = {}
    for matchup in list(matchups.values()):
        raw_matchup = matchup.copy()
        if matchup['next'] is not None:
            raw_matchup['next'] = matchup['next']['id']
        if matchup['right'] is not None:
            raw_matchup['right'] = matchup['right']['id']
        if matchup['left'] is not None:
            raw_matchup['left'] = matchup['left']['id']
        raw_matchups[raw_matchup['id']] = raw_matchup 
    return raw_matchups

tr = store_matchup_tree(matchups)
t = build_matchup_tree(tr)

In [9]:
print[(matchup['id'], matchup['start'],teams[matchup['home']]['info']['name'],teams[matchup['away']]['info']['name']) for matchup in matchups.values() if matchup['home']!=0]

[('p2', u'2016-04-15T02:30:00Z', u'Los Angeles Kings', u'San Jose Sharks'), ('p1', u'2016-04-16T02:30:00Z', u'Anaheim Ducks', u'Nashville Predators'), ('a1', u'2016-04-15T00:00:00Z', u'Florida Panthers', u'New York Islanders'), ('a2', u'2016-04-13T23:00:00Z', u'Tampa Bay Lightning', u'Detroit Red Wings'), ('m1', u'2016-04-14T23:00:00Z', u'Washington Capitals', u'Philadelphia Flyers'), ('m2', u'2016-04-14T00:00:00Z', u'Pittsburgh Penguins', u'New York Rangers'), ('c2', u'2016-04-14T01:30:00Z', u'St. Louis Blues', u'Chicago Blackhawks'), ('c1', u'2016-04-15T01:30:00Z', u'Dallas Stars', u'Minnesota Wild')]


In [42]:
matchups = create_matchups_tree()

In [31]:
#display binary tree
nb_round = 4
width = (nb_round * 2) - 1
heigh = (2**(nb_round-1)) -1
print(width,heigh)

(7, 7)


In [21]:
def display(matchups):
    nb_round = 4
    width = (nb_round * 2) - 1
    heigh = (2**(nb_round-1)) -1
    display = [['' for x in range(width)] for y in range(heigh)] 
    def walk_matchup_tree(root, x, y, dx):
        display[x][y] = root['id']
        if root['left'] is not None:
            walk_matchup_tree(root['left'], x+dx, y-(root['round']-1), dx)
        if root['right'] is not None:
            walk_matchup_tree(root['right'], x+dx, y+(root['round']-1), dx)

    display[3][2] = 'sc'
    walk_matchup_tree(matchups['w'], 2, 3, -1)
    walk_matchup_tree(matchups['e'], 4, 3, 1)

    for y in range(7):
        for x in range(7):
            id = display[x][y]
            if id != '':
                matchup = matchups[id]
                if matchup['home'] == 0:
                    sys.stdout.write('{0:15}'.format(id))
                else:
                    home = teams[matchup['home']]['info']['abbreviation']
                    away = '?'
                    if matchup['away'] != 0:
                        away = teams[matchup['away']]['info']['abbreviation']
                    sys.stdout.write('{0:3}-{2} vs {3}-{1:3}'.format(home,away,matchup['result']['home_win'],matchup['result']['away_win']))
            else:
                sys.stdout.write('{0:15}'.format(id))
        sys.stdout.write('\n')
        
display(upd._matchups)

DAL-4 vs 2-MIN                                                                           FLA-2 vs 4-NYI
               DAL-2 vs 3-STL                                             NYI-1 vs 4-TBL               
STL-4 vs 3-CHI                              sc                                           TBL-4 vs 1-DET
                              w                             TBL-0 vs 0-?                                
ANA-3 vs 4-NSH                                                                           WSH-4 vs 2-PHI
               SJS-3 vs 2-NSH                                             WSH-2 vs 3-PIT               
LAK-1 vs 4-SJS                                                                           PIT-4 vs 1-NYR


In [42]:
[(matchup['id'],matchup['left']['id']) for matchup in matchups.values() if  matchup['left'] is not None]

[('a', 'a2'),
 ('c', 'c1'),
 ('e', 'm'),
 ('m', 'm2'),
 ('p', 'p1'),
 ('w', 'c'),
 ('sc', 'w')]

In [13]:
update_matchups(matchups)

In [86]:
pprint.pprint([(matchup['id'], teams[matchup['home']]['info'], teams[matchup['away']]['info']['name'], matchup['result']) for matchup in matchups.values() if matchup['home'] != 0 and matchup['away'] != 0])

[('p2',
  {u'abbreviation': u'LAK',
   u'active': True,
   u'conference': {u'id': 5,
                   u'link': u'/api/v1/conferences/5',
                   u'name': u'Western'},
   u'division': {u'id': 15,
                 u'link': u'/api/v1/divisions/15',
                 u'name': u'Pacific'},
   u'firstYearOfPlay': u'1967',
   u'franchise': {u'franchiseId': 14,
                  u'link': u'/api/v1/franchises/14',
                  u'teamName': u'Kings'},
   u'franchiseId': 14,
   u'id': 26,
   u'link': u'/api/v1/teams/26',
   u'locationName': u'Los Angeles',
   u'name': u'Los Angeles Kings',
   u'officialSiteUrl': u'http://www.lakings.com',
   u'shortName': u'Los Angeles',
   u'teamName': u'Kings',
   u'venue': {u'city': u'Los Angeles',
              u'name': u'STAPLES Center',
              u'timeZone': {u'id': u'America/Los_Angeles', u'offset': -8}}},
  u'San Jose Sharks',
  {'away_win': 0, 'home_win': 0}),
 ('p1',
  {u'abbreviation': u'ANA',
   u'active': True,
   u'conference':

In [19]:
class Updater(object):
    
    def __init__(self, server, year):
        self._server = server
        self._year = year
        self._current_round = 0
        self._teams = {}
        self._matchups = {}
        self.load()
    
    def run(self):
        if self._current_round == 0:
            self._teams = self.get_teams()
            standings = self.get_standings(list(self._teams.values()))
            self._matchups = self.create_matchups_tree()
            self.create_matchups(standings)
            if self.is_season_finished():
                self._current_round = 1
                self.store()
        else:
            self.update_matchups()
            self.store()

    def is_season_finished(self):
        for team in list(self._teams.values()):
            remaining = 82 - team['standings']['gamesPlayed']
            if remaining > 0:
                return False
        return True

    def create_matchups(self, standings):     
        ealeader = standings['Eastern']['Atlantic'][0]
        emleader = standings['Eastern']['Metropolitan'][0]
        wcleader = standings['Western']['Central'][0]
        wpleader = standings['Western']['Pacific'][0]
        for team in standings['Eastern']['teams']:
            if int(team['standings']['wildCardRank']) == 1:
                e1wild = team
            if int(team['standings']['wildCardRank']) == 2:
                e2wild = team

        for team in standings['Western']['teams']:
            if int(team['standings']['wildCardRank']) == 1:
                w1wild = team
            if int(team['standings']['wildCardRank']) == 2:
                w2wild = team

        a1 = self._matchups['a1']
        m1 = self._matchups['m1']
        self.update_matchup(a1, ealeader['info']['id'])
        self.update_matchup(m1, emleader['info']['id'])
        if int(ealeader['standings']['conferenceRank']) < int(emleader['standings']['conferenceRank']):
            self.update_matchup(a1, e2wild['info']['id'])
            self.update_matchup(m1, e1wild['info']['id'])
        else:
            self.update_matchup(a1, e1wild['info']['id'])
            self.update_matchup(m1, e2wild['info']['id'])

        a2 = self._matchups['a2']
        self.update_matchup(a2, standings['Eastern']['Atlantic'][1]['info']['id'], standings['Eastern']['Atlantic'][2]['info']['id'])
        m2 = self._matchups['m2']
        self.update_matchup(m2, standings['Eastern']['Metropolitan'][1]['info']['id'], standings['Eastern']['Metropolitan'][2]['info']['id'])

        c1 = self._matchups['c1']
        p1 = self._matchups['p1']
        self.update_matchup(c1, wcleader['info']['id'])
        self.update_matchup(p1, wpleader['info']['id'])
        if int(wcleader['standings']['conferenceRank']) < int(wpleader['standings']['conferenceRank']):
            self.update_matchup(c1, w2wild['info']['id'])
            self.update_matchup(p1, w1wild['info']['id'])
        else:
            self.update_matchup(c1, w1wild['info']['id'])
            self.update_matchup(p1, w2wild['info']['id'])

        c2 = self._matchups['c2']
        self.update_matchup(c2, standings['Western']['Central'][1]['info']['id'], standings['Western']['Central'][2]['info']['id'])
        p2 = self._matchups['p2']
        self.update_matchup(p2, standings['Western']['Pacific'][1]['info']['id'], standings['Western']['Pacific'][2]['info']['id'])
        
    def set_matchup_childs(self, matchup, right, left):
        matchup['left'] = left
        matchup['right'] = right

    def create_matchup(self, id, round, next):
        matchup = {'id': id, 'home':0, 'away':0, 'round':round, 'start':'', 'result':{}, 'schedule':[], 'season':{}, 'next': next}
        matchup['left'] = None
        matchup['right'] = None
        matchup['result'] = {'home_win':0, 'away_win':0}
        return matchup

    def create_matchups_tree(self):
        matchups = {}
        sc = self.create_matchup('sc', 4, None)
        matchups[sc['id']] = sc

        e = self.create_matchup('e', 3, sc)
        w = self.create_matchup('w', 3, sc)
        matchups[e['id']] = e
        matchups[w['id']] = w

        a = self.create_matchup('a', 2, e)
        m = self.create_matchup('m', 2, e)
        c = self.create_matchup('c', 2, w)
        p = self.create_matchup('p', 2, w)
        matchups[a['id']] = a
        matchups[m['id']] = m
        matchups[c['id']] = c
        matchups[p['id']] = p

        a1 = self.create_matchup('a1', 1, a)
        a2 = self.create_matchup('a2', 1, a)
        m1 = self.create_matchup('m1', 1, m)
        m2 = self.create_matchup('m2', 1, m)
        c1 = self.create_matchup('c1', 1, c)
        c2 = self.create_matchup('c2', 1, c)
        p1 = self.create_matchup('p1', 1, p)
        p2 = self.create_matchup('p2', 1, p)
        matchups[a1['id']] = a1
        matchups[a2['id']] = a2
        matchups[m1['id']] = m1
        matchups[m2['id']] = m2
        matchups[c1['id']] = c1
        matchups[c2['id']] = c2
        matchups[p1['id']] = p1
        matchups[p2['id']] = p2

        #build tree
        self.set_matchup_childs(sc, e, w)

        self.set_matchup_childs(w, p, c)
        self.set_matchup_childs(e, m, a)

        self.set_matchup_childs(c, c2, c1)
        self.set_matchup_childs(p, p2, p1)
        self.set_matchup_childs(a, a2, a1)
        self.set_matchup_childs(m, m2, m1)

        return matchups

    def get_team(self, id):
        url = 'https://statsapi.web.nhl.com/api/v1/teams/' + str(id)
        team = requests.get(url).json()
        return team['teams'][0]

    def get_teams(self):
        ystr = str(self._year) + str(self._year+1)
        url = 'https://statsapi.web.nhl.com/api/v1/standings?season=' + ystr
        standings = requests.get(url).json()
        teams = {}
        for record in standings["records"]:
            for team in record['teamRecords']:
                info = self.get_team(team['team']['id'])
                team_record = {'info':info, 'standings':team, 'schedule':[]}
                teams[team['team']['id']] = team_record
        return teams

    def get_standings(self, teams):
        standings = {'Eastern':{'Atlantic':[], 'Metropolitan':[], 'teams':[]},
                     'Western':{'Central':[], 'Pacific':[], 'teams':[]},
                     'teams':[]}

        league = sorted(teams, key=lambda k: int(k['standings']['divisionRank']))
        for team in league:
            standings['teams'].append(team)
            standings[team['info']['conference']['name']]['teams'].append(team)
            standings[team['info']['conference']['name']][team['info']['division']['name']].append(team)
        standings['teams'] = sorted(standings['teams'], key=lambda k: int(k['standings']['leagueRank']))

        standings['Eastern']['teams'] = sorted(standings['Eastern']['teams'], key=lambda k: int(k['standings']['conferenceRank']))
        standings['Western']['teams'] = sorted(standings['Western']['teams'], key=lambda k: int(k['standings']['conferenceRank']))

        standings['Eastern']['Atlantic'] = sorted(standings['Eastern']['Atlantic'], key=lambda k: int(k['standings']['divisionRank']))
        standings['Eastern']['Metropolitan'] = sorted(standings['Eastern']['Metropolitan'], key=lambda k: int(k['standings']['divisionRank']))
        standings['Western']['Central'] = sorted(standings['Western']['Central'], key=lambda k: int(k['standings']['divisionRank']))
        standings['Western']['Pacific'] = sorted(standings['Western']['Pacific'], key=lambda k: int(k['standings']['divisionRank']))

        return standings

    def parse_time(self, timestamp):
        from_zone = tz.gettz('UTC')
        to_zone = tz.gettz('America/New_York')
        utc = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%SZ')
        utc = utc.replace(tzinfo=from_zone)
        return utc.astimezone(to_zone)

    def get_matchup_schedule(self, matchup, schedules=None):
        home_id = matchup['home']
        away_id = matchup['away']
        result = []
        s = schedules
        if schedules is None:
            s = get_playoff_schedule(int(home_id), year)
        for date in s['dates']:
            game = date['games'][0]
            game_home_id = game['teams']['home']['team']['id']
            game_away_id = game['teams']['away']['team']['id']
            if game['gameType'] == 'P':
                if game_home_id == away_id or game_away_id == away_id:
                    result.append(game)
        result = sorted(result, key=lambda k: parse_time(k['gameDate']))
        return result

    def get_matchup_start(self, matchup):
        if len(matchup['schedule']) == 0:
            return ''
        return matchup['schedule'][0]['gameDate']

    def get_matchup_season_result(self, home, away):
        result = {'home_win':0, 'away_win':0, 'matchs':[]}
        schedule = self._teams[home]['schedule']
        if len(schedule) == 0:
            schedule = self.get_schedule(home)
            self._teams[home]['schedule'] = schedule

        for date in schedule['dates']:
            game = date['games'][0]
            game_home_id = game['teams']['home']['team']['id']
            game_away_id = game['teams']['away']['team']['id']
            if game_home_id == away:
                print(game['gameDate'],game['teams']['away']['score'],game['teams']['home']['score'])
                if int(game['teams']['home']['score']) > int(game['teams']['away']['score']):
                    result['away_win'] = result['away_win'] + 1
                elif int(game['teams']['home']['score']) < int(game['teams']['away']['score']):
                    result['home_win'] = result['home_win'] + 1
                result['matchs'].append({'home': int(game['teams']['away']['score']), 'away': int(game['teams']['home']['score'])})
            if game_away_id == away:
                print(game['gameDate'],game['teams']['home']['score'],game['teams']['away']['score'])
                if int(game['teams']['home']['score']) > int(game['teams']['away']['score']):
                    result['home_win'] = result['home_win'] + 1
                elif int(game['teams']['home']['score']) < int(game['teams']['away']['score']):
                    result['away_win'] = result['away_win'] + 1
                result['matchs'].append({'home': int(game['teams']['home']['score']), 'away': int(game['teams']['away']['score'])})
        return result
    
    def get_matchup_result(self, matchup):
        result = {}
        home_id = matchup['home']
        away_id = matchup['away']
        home_win = 0
        away_win = 0
        for game in matchup['schedule']:
            game_home_id = game['teams']['home']['team']['id']
            game_away_id = game['teams']['away']['team']['id']
            if game['gameType'] == 'P':
                if game_home_id == away_id or game_away_id == away_id:
                    if game_home_id == away_id: #reverse
                        away_score = game['teams']['home']['score']
                        home_score = game['teams']['away']['score']
                    else:
                        away_score = game['teams']['away']['score']
                        home_score = game['teams']['home']['score']
                    if home_score > away_score:
                        home_win = home_win + 1
                    elif home_score < away_score:
                        away_win = away_win + 1
                    #print(match['home']['team']['name'], home_score, match['away']['team']['name'], away_score)
                    #print(game['teams']['home']['team']['name'],game['teams']['home']['score'],match['away']['team']['name'],game['teams']['away']['score'])
        result['home_win'] = home_win
        result['away_win'] = away_win
        return result

    def is_matchup_finished(self, matchup):
        return matchup['result']['home_win'] == 4 or matchup['result']['away_win'] == 4

    def get_matchup_winner(self, matchup):
        if matchup['result']['home_win'] == 4:
            return matchup['home']
        if matchup['result']['away_win'] == 4:
            return matchup['away']
        return 0

    def update_matchup(self, matchup, home=0, away=0):
        if is_matchup_finished(matchup):
            return

        if matchup['home'] != 0 and matchup['away'] != 0:
            #update result and maybe pass to next stage
            matchup['schedule'] = get_matchup_schedule(matchup, year)
            if matchup['start'] == '':
                matchup['start'] = get_matchup_start(matchup)
            matchup['result'] = get_matchup_result(matchup)
            if is_matchup_finished(matchup) and matchup['next'] is not None:
                print('Finished',matchup['id'])
                update_matchup(matchup['next'], get_matchup_winner(matchup))
        else:
            if matchup['home'] == 0:
                matchup['home'] = home
                matchup['away'] = away
            else:
                matchup['away'] = home

            if matchup['home'] != 0 and matchup['away'] != 0:
                #Begin matchup
                hi = teams[matchup['home']]
                ai = teams[matchup['away']]
                if int(hi['standings']['leagueRank']) > int(ai['standings']['leagueRank']): 
                    matchup['home'] = ai['info']['id']
                    matchup['away'] = hi['info']['id']
                hi = teams[matchup['home']]
                ai = teams[matchup['away']]
                matchup['season'] = self.get_matchup_season_result(matchup['home'], matchup['away'])
                matchup['schedule'] = get_matchup_schedule(matchup, year)
                matchup['start'] = get_matchup_start(matchup)

    def update_matchups(self):
        ms = list(self._matchups.values())
        ms = sorted(ms, key=lambda k: k['round'])
        for matchup in ms:
            self.update_matchup(matchup)

    def get_schedule(self, team):
        print('Get schedule for ' + str(team))
        url = 'https://statsapi.web.nhl.com/api/v1/schedule?startDate=' + str(self._year) + '-10-01&endDate=' + str(self._year+1) + '-06-29&expand=schedule.teams,schedule.linescore,schedule.broadcasts,schedule.ticket,schedule.game.content.media.epg&leaderCategories=&site=en_nhlCA&teamId=' + str(team)
        team_schedule = requests.get(url)
        return team_schedule.json()

    def get_playoff_schedule(self, team):
        url = 'https://statsapi.web.nhl.com/api/v1/schedule?startDate=' + str(self._year+1) + '-04-01&endDate=' + str(self._year+1) + '-06-29&expand=schedule.teams,&site=en_nhlCA&teamId=' + str(team)
        team_schedule = requests.get(url)
        return team_schedule.json()

    def fetch_data(self):
        data = requests.get('http://' + self._server + '/nhlplayoffs/api/v3.0/' + str(self._year) + '/data').json()
        return data

    def update_data(self, data):
        url = 'http://' + self._server + '/nhlplayoffs/api/v3.0/' + str(self._year) + '/data'
        headers = {'content-type': 'application/json'}
        requests.post(url, data = json.dumps(data), headers=headers)
    
    def build_matchup_tree(self, raw_matchups):
        matchups = {}
        for matchup_raw in list(raw_matchups.values()):
            matchup = matchup_raw.copy()
            matchups[matchup['id']] = matchup
        
        for matchup in list(matchups.values()):
            next = matchup['next']
            right = matchup['right']
            left = matchup['left']
            if next in raw_matchups:
                matchup['next'] = matchups[next]
            if right in raw_matchups:
                matchup['right'] = matchups[right]
            if left in raw_matchups:
                matchup['left'] = matchups[left]
        return matchups

    def store_matchup_tree(self, matchups):
        raw_matchups = {}
        for matchup in list(matchups.values()):
            raw_matchup = matchup.copy()
            if matchup['next'] is not None:
                raw_matchup['next'] = matchup['next']['id']
            if matchup['right'] is not None:
                raw_matchup['right'] = matchup['right']['id']
            if matchup['left'] is not None:
                raw_matchup['left'] = matchup['left']['id']
            raw_matchups[raw_matchup['id']] = raw_matchup 
        return raw_matchups
        
    def load(self):
        data = self.fetch_data()
        self._teams = data['teams']
        self._current_round = data['current_round']
        self._matchups = self.build_matchup_tree(data['matchups'])
        
    def store(self):
        data = {}
        data['teams'] = self._teams
        data['current_round'] = self._current_round
        data['matchups'] = self.store_matchup_tree(self._matchups)
        self.update_data(data)
        

In [20]:
upd = Updater('localhost:5000', 2015)
upd.run()

('Finished', u'p2')
('Finished', u'p1')
('Finished', u'a1')
('Finished', u'a2')
('Finished', u'm1')
('Finished', u'm2')
('Finished', u'c2')
('Finished', u'c1')
('Finished', u'a')


In [82]:
[(matchup['id']) for matchup in t.values() if type(matchup['left']) is str]

[]

In [41]:
t=matchups.copy()
t['a1']['next'] = 'ds'
print(matchups['a1']['next'])

ds


In [14]:
upd._matchups['a1']['home']

13

In [26]:
def is_season_finished(self):
    for team in list(self._teams.values()):
        remaining = 82 - team['standings']['gamesPlayed']
        if remaining > 0:
            return False
    return True

In [27]:
is_season_finished(upd)

True

In [12]:
def get_matchup_season_result(self, home, away):
        result = {'home_win':0, 'away_win':0, 'matchs':[]}
        schedule = self._teams[home]['schedule']
        if len(schedule) == 0:
            schedule = self.get_schedule(home)
            self._teams[home]['schedule'] = schedule

        for date in schedule['dates']:
            game = date['games'][0]
            game_home_id = game['teams']['home']['team']['id']
            game_away_id = game['teams']['away']['team']['id']
            if game_home_id == away:
                print(game['gameDate'],game['teams']['away']['score'],game['teams']['home']['score'])
                if int(game['teams']['home']['score']) > int(game['teams']['away']['score']):
                    result['away_win'] = result['away_win'] + 1
                elif int(game['teams']['home']['score']) < int(game['teams']['away']['score']):
                    result['home_win'] = result['home_win'] + 1
                result['matchs'].append({'home': int(game['teams']['away']['score']), 'away': int(game['teams']['home']['score'])})
            if game_away_id == away:
                print(game['gameDate'],game['teams']['home']['score'],game['teams']['away']['score'])
                if int(game['teams']['home']['score']) > int(game['teams']['away']['score']):
                    result['home_win'] = result['home_win'] + 1
                elif int(game['teams']['home']['score']) < int(game['teams']['away']['score']):
                    result['away_win'] = result['away_win'] + 1
                result['matchs'].append({'home': int(game['teams']['home']['score']), 'away': int(game['teams']['away']['score'])})
        return result

In [15]:
result = get_matchup_season_result(upd, upd._matchups['a1']['home'], upd._matchups['a1']['away'])

Get schedule for 13
(u'2015-11-28T00:30:00Z', 3, 2)
(u'2015-12-16T00:00:00Z', 5, 1)
(u'2016-03-14T23:00:00Z', 2, 3)
(u'2016-04-15T00:00:00Z', 4, 5)
(u'2016-04-15T23:30:00Z', 3, 1)
(u'2016-04-18T00:00:00Z', 3, 4)
(u'2016-04-21T00:00:00Z', 2, 1)
(u'2016-04-23T00:00:00Z', 1, 2)
(u'2016-04-24T23:00:00Z', 1, 2)


In [18]:
upd._teams[13]['schedule']

{u'copyright': u'NHL and the NHL Shield are registered trademarks of the National Hockey League. NHL and NHL team marks are the property of the NHL and its teams. \xa9 NHL 2016. All Rights Reserved.',
 u'dates': [{u'date': u'2015-10-10',
   u'games': [{u'content': {u'editorial': {},
      u'highlights': {},
      u'link': u'/api/v1/game/2015020020/content',
      u'media': {u'epg': [{u'items': [{u'callLetters': u'',
           u'eventId': u'221-1000123',
           u'feedName': u'',
           u'freeGame': False,
           u'gamePlus': False,
           u'language': u'',
           u'mediaFeedType': u'HOME',
           u'mediaPlaybackId': u'42415303',
           u'mediaState': u'MEDIA_ARCHIVE'}],
         u'platform': u'web',
         u'title': u'NHLTV'},
        {u'items': [], u'title': u'Audio'},
        {u'items': [], u'title': u'Extended Highlights', u'topicList': u''},
        {u'items': [], u'title': u'Recap', u'topicList': u''}]}},
     u'gameDate': u'2015-10-10T23:00:00Z',
   

In [17]:
upd.store()